In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import random
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

In [2]:
df = pd.read_excel('../data/GNP_Aerial_counting_1969_2022.xlsx')

In [3]:
empty_cols = ['MALE', 'CALVES'] #columns that are empty
zero_cols = ['LINE2002', 'LINE2012', 'COLLAR', 'CONSERVANC', 'SANCTUARY'] #columns that are > 80% just 0s
drop_cols = ['NOTES'] # other columns to drop
df.drop(columns=empty_cols, inplace=True)
df.drop(columns=zero_cols, inplace=True)
df.drop(columns=drop_cols, inplace=True)

In [4]:
df['TIME'] = df['TIME'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second if pd.notna(x) else x)
df['TIME'] = df['TIME'].fillna(0)

In [5]:
df['TYPE'] = df['TYPE'].map({'Fixed-wing': 0, 'Helicopter': 1})

In [14]:
#zero_count = (df['COUNT_DAY'] == 0).sum()
#print(zero_count / df.shape[0])

In [6]:
df['DATE'] = df['DATE'].apply(lambda t: t.day if isinstance(t, datetime) else np.nan)
df['DATE'] = pd.to_numeric(df['DATE'], errors='coerce')

In [7]:
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

df['MONTH'] = df['MONTH'].map(month_mapping)
df['MONTH'] = pd.to_numeric(df['MONTH'], errors='coerce')

In [8]:
df['lat_lag1'] = df.groupby('SPECIES')['LATITUDE'].shift(1)
df['lon_lag1'] = df.groupby('SPECIES')['LONGITUDE'].shift(1)
df['lat_lag2'] = df.groupby('SPECIES')['LATITUDE'].shift(2)
df['lon_lag2'] = df.groupby('SPECIES')['LONGITUDE'].shift(2)
df['count_lag1'] = df.groupby('SPECIES')['COUNT_DAY'].shift(1)
df['count_lag2'] = df.groupby('SPECIES')['COUNT_DAY'].shift(2)

In [9]:
df['SPECIES'] = df['SPECIES'].str.lower()
df['STRATUM'] = df['STRATUM'].str.lower()
df = pd.get_dummies(df, columns=['SPECIES', 'STRATUM'])

In [10]:
'''
correlation_matrix = df.corr()
plt.figure(figsize=(25, 25)) 
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix Heatmap")
plt.show()
'''

'\ncorrelation_matrix = df.corr()\nplt.figure(figsize=(25, 25)) \nsns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")\nplt.title("Correlation Matrix Heatmap")\nplt.show()\n'

### Non-DL Model

#### Run Cross-Val

In [22]:
def run_cv(df, num_splits=5):
    tested_years = [2022]
    ave_mse = 0
    ave_r2 = 0

    for _ in range(num_splits):
        cv_year = random.choice(df['COUNT'].unique())
        while cv_year in tested_years:
            cv_year = random.choice(df['COUNT'].unique())
        
        tested_years.append(cv_year)

        train_df = df[~df['COUNT'].isin([2022, cv_year])]
        test_df = df[df['COUNT'] == cv_year]

        #fillna with mean
        train_df = train_df.fillna(train_df.mean())
        test_df = test_df.fillna(test_df.mean())

        X_train = train_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'COUNT_DAY'])
        y_train = train_df[['COUNT_DAY', 'LATITUDE', 'LONGITUDE']]
        X_test = test_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'COUNT_DAY'])
        y_test = test_df[['COUNT_DAY', 'LATITUDE', 'LONGITUDE']]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Multi-output regressor
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Predictions
        y_pred = model.predict(X_test)

        print(f"year being tested for CV: {cv_year}")

        mse = mean_squared_error(y_test, y_pred)
        print(f"Mean Squared Error: {mse}")

        # Example: R² for predictions
        r2 = r2_score(y_test, y_pred)
        print(f"R² Score: {r2}")

        ave_mse += mse
        ave_r2 += r2

    print(f"Average Mean Squared Error: {ave_mse / num_splits}")
    print(f"Average R² Score: {ave_r2 / num_splits}")

run_cv(df)

year being tested for CV: 2002
Mean Squared Error: 0.011390783293928121
R² Score: 0.766421487847372
year being tested for CV: 2012
Mean Squared Error: 0.448780217578266
R² Score: 0.7319068516867674
year being tested for CV: 2018
Mean Squared Error: 0.012596944950188373
R² Score: 0.7797733530405973
year being tested for CV: 1972
Mean Squared Error: 0.004966876743004004
R² Score: 0.7389944974429555
year being tested for CV: 2004
Mean Squared Error: 0.05149120966480534
R² Score: 0.22744298911616698
Average Mean Squared Error: 0.10584520644603837
Average R² Score: 0.6489078358267719


#### Train on Full Dataset (Excluding 2022 for testing)

In [12]:
# Get Train Test Split
train_df = df[df['COUNT'] != 2022]
test_df = df[df['COUNT'] == 2022]

#fillna with mean
train_df = train_df.fillna(train_df.mean())
test_df = test_df.fillna(test_df.mean())

X_train = train_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'COUNT_DAY'])
y_train = train_df[['COUNT_DAY', 'LATITUDE', 'LONGITUDE']]
X_test = test_df.drop(columns=['ID', 'LATITUDE', 'LONGITUDE', 'COUNT_DAY'])
y_test = test_df[['COUNT_DAY', 'LATITUDE', 'LONGITUDE']]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Multi-output regressor
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Example: Calculate MSE for your model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Example: R² for predictions
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

Mean Squared Error: 0.008737311556722062
R² Score: 0.9444638811245224


### Set-Up NN

In [13]:
# Example: Convert features and target to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [14]:
# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
# Define the neural network
class MultiOutputNN(nn.Module):
    def __init__(self, input_size, output_size=3):
        super(MultiOutputNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32) 
        self.output = nn.Linear(32, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.output(x)
        return x

### Run 1 NN

In [16]:
model = MultiOutputNN(input_size=X_train_tensor.shape[1])

In [17]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 100

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/100, Loss: 58.2837
Epoch 2/100, Loss: 0.3902
Epoch 3/100, Loss: 0.2790
Epoch 4/100, Loss: 0.2228
Epoch 5/100, Loss: 0.1812
Epoch 6/100, Loss: 0.1587
Epoch 7/100, Loss: 0.1428
Epoch 8/100, Loss: 0.1284
Epoch 9/100, Loss: 0.1153
Epoch 10/100, Loss: 0.1052
Epoch 11/100, Loss: 0.0942
Epoch 12/100, Loss: 0.0864
Epoch 13/100, Loss: 0.0806
Epoch 14/100, Loss: 0.0718
Epoch 15/100, Loss: 0.0690
Epoch 16/100, Loss: 0.0634
Epoch 17/100, Loss: 0.0595
Epoch 18/100, Loss: 0.0595
Epoch 19/100, Loss: 0.0526
Epoch 20/100, Loss: 0.0509
Epoch 21/100, Loss: 0.0484
Epoch 22/100, Loss: 0.0460
Epoch 23/100, Loss: 0.0453
Epoch 24/100, Loss: 0.0442
Epoch 25/100, Loss: 0.0428
Epoch 26/100, Loss: 0.0395
Epoch 27/100, Loss: 0.0391
Epoch 28/100, Loss: 0.0373
Epoch 29/100, Loss: 0.0377
Epoch 30/100, Loss: 0.0349
Epoch 31/100, Loss: 0.0354
Epoch 32/100, Loss: 0.0337
Epoch 33/100, Loss: 0.0329
Epoch 34/100, Loss: 0.0318
Epoch 35/100, Loss: 0.0317
Epoch 36/100, Loss: 0.0311
Epoch 37/100, Loss: 0.0307
Epoch 38/

In [18]:
with torch.no_grad():
    actual = y_test_tensor.numpy()
    y_preds = model(X_test_tensor)
    #y_preds[:, 0] = torch.round(y_preds[:, 0])
    predicted = y_preds.detach().numpy()
    
    mse = mean_squared_error(actual, predicted)
    print(f"Mean Squared Error: {mse}")

    mae = mean_absolute_error(actual, predicted)
    print(f"Mean Absolute Error: {mae}")    

    r2 = r2_score(actual, predicted)
    print(f"R² Score: {r2}")

Mean Squared Error: 0.22942012548446655
Mean Absolute Error: 0.21979950368404388
R² Score: 0.8434914946556091


### Ensemble NNs (So far better than 1)

In [19]:
models = [MultiOutputNN(input_size=X_train.shape[1]) for _ in range(3)]

In [20]:
criterion = nn.MSELoss()
epochs = 100

trained_models = []
for i, model in enumerate(models):
    tot_loss = 0.0
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        tot_loss += running_loss / len(train_loader)

    print(f"Loss for Model {i+1}: {tot_loss / 100:.4f}")
    trained_models.append(model)

Loss for Model 1: 0.6375
Loss for Model 2: 0.6034
Loss for Model 3: 0.6245


In [21]:
# Predict with all models
predictions = []

for model in trained_models:
    model.eval()
    with torch.no_grad():
        y_preds = model(X_test_tensor)
        predictions.append(y_preds.detach().numpy())

# Average predictions
predicted = sum(predictions) / len(predictions)

actual = y_test_tensor.numpy()
mse = mean_squared_error(actual, predicted)
print(f"Mean Squared Error: {mse}")

mae = mean_absolute_error(actual, predicted)
print(f"Mean Absolute Error: {mae}")    

r2 = r2_score(actual, predicted)
print(f"R² Score: {r2}")

Mean Squared Error: 0.009994489140808582
Mean Absolute Error: 0.05950450524687767
R² Score: 0.9502697587013245
